In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

In [2]:
def extract_fishing_points(url):
    """
    지도 사이트에서 낚시 포인트 이름을 추출하는 함수
    
    Args:
        url (str): 낚시 포인트 지도가 있는 웹사이트 URL
    
    Returns:
        list: 낚시 포인트 이름 리스트
    """
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저 창 표시 없이 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # WebDriver 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    fishing_points = []
    
    try:
        # 웹페이지 로드
        driver.get(url)
        print("페이지 로딩 중...")
        
        # 페이지가 완전히 로드될 때까지 잠시 대기
        time.sleep(3)
        
        # 지도가 완전히 로드될 때까지 기다림 (예: 특정 요소가 나타날 때까지)
        wait = WebDriverWait(driver, 20)
        
        # 지도가 로드될 때까지 기다림
        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.label"))
        )
        
        # 축소 버튼 찾기 (title="축소" 및 type="button" 속성을 가진 버튼)
        try:
            zoom_out_button = driver.find_element(By.XPATH, '//button[@title="축소" and @type="button"]')
            
            # 최대한 축소하기 위해 여러 번 클릭 (10번 정도면 최대 축소에 충분할 것)
            print("지도 최대 축소 중...")
            for _ in range(10):
                try:
                    zoom_out_button.click()
                    time.sleep(0.5)  # 각 클릭 사이에 잠시 대기
                except Exception as e:
                    print(f"더 이상 축소할 수 없습니다: {e}")
                    break
            
            print("지도 축소 완료")
            
            # 지도가 업데이트될 때까지 잠시 대기
            time.sleep(2)
            
        except Exception as e:
            print(f"축소 버튼을 찾을 수 없습니다: {e}")
        
        # div(class="label") 안에 span(class="center") 요소 찾기
        point_elements = driver.find_elements(By.CSS_SELECTOR, "div.label span.center")
        
        # 각 포인트 이름 추출
        for point in point_elements:
            point_name = point.text.strip()
            if point_name:  # 빈 문자열이 아닌 경우만 추가
                fishing_points.append({"name": point_name})
        
        print(f"총 {len(fishing_points)}개의 낚시 포인트를 찾았습니다.")
        
    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")
    
    finally:
        # WebDriver 종료
        driver.quit()
    
    return fishing_points

def save_to_csv(fishing_points, filename):
    try:
        filename = filename + ".csv"
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['name']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for point in fishing_points:
                writer.writerow(point)
                
        print(f"{len(fishing_points)}개의 낚시 포인트 정보를 {filename}에 저장했습니다.")
        
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")


# 낚시 포인트 추출 실행
west_sea_url = "https://m.badatime.com/time_map.jsp?idx=121&menu=2"
south_sea_url = "https://m.badatime.com/time_map.jsp?idx=41&menu=2"
east_sea_url = "https://m.badatime.com/time_map.jsp?idx=203&menu=2"
jeju_sea_url = "https://m.badatime.com/time_map.jsp?idx=67&menu=2"

west_sea_points = extract_fishing_points(west_sea_url)

south_sea_points = extract_fishing_points(south_sea_url)

east_sea_points = extract_fishing_points(east_sea_url)

jeju_sea_points = extract_fishing_points(jeju_sea_url)

페이지 로딩 중...


KeyboardInterrupt: 

In [3]:
fishing_points = west_sea_points + east_sea_points + south_sea_points + jeju_sea_points
print(len(fishing_points))
save_to_csv(fishing_points, filename="fishing_points")

NameError: name 'west_sea_points' is not defined

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import requests
import csv
import time
import json
import os
import dotenv

def load_fishing_points(filename):
    """
    CSV 파일에서 낚시 포인트 이름을 로드하는 함수
    
    Args:
        filename (str): 낚시 포인트 이름이 저장된 CSV 파일 경로
    
    Returns:
        list: 낚시 포인트 이름 리스트
    """
    fishing_points = []
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if 'name' in row and row['name']:
                    fishing_points.append(row['name'])
        
        print(f"{len(fishing_points)}개의 낚시 포인트 이름을 로드했습니다.")
        
    except Exception as e:
        print(f"CSV 파일 로드 중 오류 발생: {e}")
    
    return fishing_points

def load_coastline_shapefile(shapefile_path):
    """
    해안선 Shapefile을 로드하는 함수
    
    Args:
        shapefile_path (str): Shapefile 경로 (.shp 확장자)
    
    Returns:
        geopandas.GeoDataFrame: 해안선 데이터
    """
    try:
        # Shapefile 로드
        coastline_gdf = gpd.read_file(shapefile_path)
        
        # 좌표계 정보 확인 및 출력
        crs = coastline_gdf.crs
        print(f"해안선 데이터 좌표계: {crs}")
        
        # WGS84 좌표계(EPSG:4326)로 변환 (필요한 경우)
        if crs != "EPSG:4326":
            print(f"좌표계를 WGS84(EPSG:4326)로 변환합니다...")
            coastline_gdf = coastline_gdf.to_crs("EPSG:4326")
        
        # 데이터 기본 정보 출력
        print(f"해안선 데이터 로드 완료: {len(coastline_gdf)} 개의 해안선 객체")
        print(f"컬럼 목록: {coastline_gdf.columns.tolist()}")
        
        return coastline_gdf
    
    except Exception as e:
        print(f"Shapefile 로드 중 오류 발생: {e}")
        return None

def create_coastline_buffer(coastline_gdf, buffer_distance_km=2.0):
    """
    해안선 데이터에 버퍼를 생성하는 함수 (해안선으로부터 일정 거리 이내 영역)
    
    Args:
        coastline_gdf (geopandas.GeoDataFrame): 해안선 데이터
        buffer_distance_km (float): 버퍼 거리 (km)
    
    Returns:
        geopandas.GeoDataFrame: 버퍼가 적용된 해안선 데이터
    """
    try:
        # 먼저 투영 좌표계로 변환 (거리 계산을 위해)
        # 한국에 적합한 UTM-K (EPSG:5179) 좌표계 사용
        projected_gdf = coastline_gdf.to_crs("EPSG:5179")
        
        # 버퍼 적용 (미터 단위)
        buffer_distance_m = buffer_distance_km * 1000
        buffer_gdf = projected_gdf.buffer(buffer_distance_m)
        
        # 버퍼를 GeoDataFrame으로 변환
        buffer_gdf = gpd.GeoDataFrame(geometry=buffer_gdf, crs="EPSG:5179")
        
        # WGS84로 다시 변환
        buffer_gdf = buffer_gdf.to_crs("EPSG:4326")
        
        print(f"해안선으로부터 {buffer_distance_km}km 이내 버퍼 영역 생성 완료")
        return buffer_gdf
    
    except Exception as e:
        print(f"버퍼 생성 중 오류 발생: {e}")
        return None

def is_near_coastline(point_lat, point_lon, coastline_buffer_gdf):
    """
    주어진 좌표가 해안선 버퍼 영역 내에 있는지 확인하는 함수
    
    Args:
        point_lat (float): 위도
        point_lon (float): 경도
        coastline_buffer_gdf (geopandas.GeoDataFrame): 해안선 버퍼 데이터
    
    Returns:
        bool: 해안선 근처인 경우 True, 아니면 False
    """
    try:
        # 포인트 생성
        point = Point(point_lon, point_lat)  # GIS에서는 (경도, 위도) 순서로 사용
        
        # GeoDataFrame으로 변환
        point_gdf = gpd.GeoDataFrame(geometry=[point], crs="EPSG:4326")
        
        # 공간 조인으로 버퍼 영역 내에 있는지 확인
        # 하나 이상의 버퍼 영역과 교차하면 해안선 근처로 판단
        for buffer_geom in coastline_buffer_gdf.geometry:
            if point_gdf.geometry[0].intersects(buffer_geom):
                return True
        
        return False
    
    except Exception as e:
        print(f"해안선 근접 확인 중 오류 발생: {e}")
        return False

def get_coordinates_from_kakao(place_name, api_key):
    """
    카카오맵 API를 사용하여 장소 이름으로 좌표를 검색하는 함수
    
    Args:
        place_name (str): 검색할 장소 이름
        api_key (str): 카카오 REST API 키
    
    Returns:
        dict: 검색 결과 (이름, 위도, 경도, 주소 등)
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {
        "query": place_name,
        "size": 15
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("documents") and len(result["documents"]) > 0:
                # 검색 결과 중 첫 번째 항목 사용
                first_result = result["documents"][0]
                
                return {
                    "name": place_name,
                    "latitude": float(first_result["y"]),
                    "longitude": float(first_result["x"]),
                    "address": first_result.get("address_name", ""),
                    "place_name": first_result.get("place_name", ""),
                    "category": first_result.get("category_name", ""),
                    "phone": first_result.get("phone", ""),
                    "place_url": first_result.get("place_url", "")
                }
        else:
            print(f"API 요청 실패: {response.status_code}, {response.text}")
    
    except Exception as e:
        print(f"{place_name} 검색 중 오류: {e}")
    
    return None

def is_coastal_location(location_name):
    """
    위치 이름을 기반으로 해안가일 가능성을 판단하는 함수
    
    Args:
        location_name (str): 위치 이름
    
    Returns:
        bool: 해안가일 가능성이 높으면 True, 아니면 False
    """
    coastal_keywords = [
        '항', '포', '도', '진', '해변', '해수욕장', '바다', '어촌', 
        '해안', '갯벌', '선착장', '방파제', '등대', '요트', '마리나',
        '배', '페리', '여객터미널', '수산', '어판장', '어시장', '수협',
        '바닷가', '해', '부두', '간석지', '갑문', '등표', '방조제'
    ]
    
    for keyword in coastal_keywords:
        if keyword in location_name:
            return True
    
    return False

def process_fishing_points_with_shapefile(input_filename, output_filename, shapefile_path, kakao_api_key, buffer_distance_km=5.0):
    """
    Shapefile을 활용하여 낚시 포인트 이름을 좌표로 변환하고 바다 근처 포인트만 필터링하는 함수
    
    Args:
        input_filename (str): 낚시 포인트 이름이 저장된 CSV 파일 경로
        output_filename (str): 결과를 저장할 CSV 파일 경로
        shapefile_path (str): 해안선 Shapefile 경로
        kakao_api_key (str): 카카오 REST API 키
        buffer_distance_km (float): 해안선으로부터 최대 거리 (km)
    """
    # 낚시 포인트 이름 로드
    fishing_points = load_fishing_points(input_filename)
    
    if not fishing_points:
        print("처리할 낚시 포인트가 없습니다.")
        return
    
    # 해안선 데이터 로드
    coastline_gdf = load_coastline_shapefile(shapefile_path)
    if coastline_gdf is None:
        print("해안선 데이터를 로드할 수 없습니다.")
        return
    
    # 해안선 버퍼 생성
    coastline_buffer = create_coastline_buffer(coastline_gdf, buffer_distance_km)
    if coastline_buffer is None:
        print("해안선 버퍼를 생성할 수 없습니다.")
        return
    
    # 결과 저장할 리스트
    results = []
    
    # 각 낚시 포인트에 대해 좌표 검색 및 해안선 근처 확인
    for i, point_name in enumerate(fishing_points):
        print(f"[{i+1}/{len(fishing_points)}] '{point_name}' 처리 중...")
        
        # 검색 키워드 준비
        search_keywords = [point_name]
        
        # 키워드가 없는 경우에만 추가 검색어 생성
        if not is_coastal_location(point_name):
            search_keywords.extend([
                point_name + " 낚시",
                point_name + " 포구",
                point_name + " 해변",
                point_name + " 항구"
            ])
        
        # 검색어 목록을 순회하며 좌표 찾기
        result = None
        for keyword in search_keywords:
            result = get_coordinates_from_kakao(keyword, kakao_api_key)
            if result:
                result["name"] = point_name  # 원래 이름으로 복원
                result["search_keyword"] = keyword  # 사용된 검색어 저장
                break
        
        if result:
            # 해안선 근처인지 확인
            lat = result["latitude"]
            lon = result["longitude"]
            
            is_near_coast = is_near_coastline(lat, lon, coastline_buffer)
            result["near_coastline"] = is_near_coast
            
            if is_near_coast:
                print(f"  ✓ 해안선 근처 포인트: {point_name} ({lat}, {lon})")
                results.append(result)
            else:
                print(f"  ✗ 해안선에서 멀리 떨어짐: {point_name}")
        else:
            print(f"  ! 좌표를 찾을 수 없음: {point_name}")
        
        # API 요청 제한 고려
        time.sleep(0.2)
    
    # 결과를 CSV 파일로 저장
    if results:
        try:
            # 모든 결과의 키 수집
            all_keys = set()
            for result in results:
                all_keys.update(result.keys())
            
            fieldnames = list(all_keys)
            
            with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                
                writer.writeheader()
                for result in results:
                    writer.writerow(result)
            
            print(f"\n총 {len(results)}개의 해안선 근처 낚시 포인트를 '{output_filename}'에 저장했습니다.")
            
        except Exception as e:
            print(f"CSV 파일 저장 중 오류 발생: {e}")
    else:
        print("\n해안선 근처의 낚시 포인트를 찾지 못했습니다.")

In [6]:

def main():
    # 입력 및 출력 파일 경로
    input_file = "/home/chichi/S12P21C201/BE/fish_points/fishing_points.csv"  # 낚시 포인트 이름이 저장된 CSV
    output_file = "/home/chichi/S12P21C201/BE/fish_points/fishing_points_coastline_2km.csv"  # 결과 저장할 CSV
    
    # 해안선 Shapefile 경로
    shapefile_path = "/home/chichi/S12P21C201/BE/fish_points/coastline_data/해안선데이터.shp"
    
    # 카카오 API 키 설정
    dotenv.load_dotenv()
    kakao_api_key = os.environ.get("KAKAO_API_KEY")
    if not kakao_api_key:
        print("⚠️ 카카오맵 API 키를 설정해주세요!")
        print("1. https://developers.kakao.com 에서 앱을 등록하고 API 키를 발급받으세요.")
        print("2. 발급받은 REST API 키를 코드에 입력하거나 KAKAO_API_KEY 환경 변수로 설정하세요.")
        return
    
    # 처리 시작
    print("Shapefile을 활용하여 낚시 포인트 이름을 좌표로 변환하고 해안선 근처 포인트만 필터링합니다...")
    process_fishing_points_with_shapefile(input_file, output_file, shapefile_path, kakao_api_key)

if __name__ == "__main__":
    main()

Shapefile을 활용하여 낚시 포인트 이름을 좌표로 변환하고 해안선 근처 포인트만 필터링합니다...
1415개의 낚시 포인트 이름을 로드했습니다.
해안선 데이터 좌표계: PROJCS["Korea_2000_Korea_Central_Belt_2010",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
좌표계를 WGS84(EPSG:4326)로 변환합니다...
해안선 데이터 로드 완료: 82118 개의 해안선 객체
컬럼 목록: ['GRP_SGG', 'GRP_ISL', 'GRP_CON', 'GRP_STA', 'CAT_COA', 'CAT_SUR', 'SOR_DAT', 'NAM_OBJ', 'SGG_NAM', 'INF_IND', 'geometry']
해안선으로부터 5.0km 이내 버퍼 영역 생성 완료
[1/1415] '진도' 처리 중...
  ✗ 해안선에서 멀리 떨어짐: 진도
[2/1415] '어란진항' 처리 중...
  ✓ 해안선 근처 포인트: 어란진항 (34.3505957827997, 126.475683551779)
[3/1415] '서망항' 처